In [8]:
from langchain.chat_models import init_chat_model 
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnableMap
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import time
from IPython.display import display, Markdown
load_dotenv()

True

In [9]:
model = init_chat_model(model="deepseek/deepseek-chat-v3-0324:free",model_provider="openai") # initializing chat model

In [10]:
print(model.invoke([{"role":"system" , "content": "what language do you think the user is speaking in?"} , {"role": "user", "content": "tussi ki kr rhe ho"}]).content) # low level model prompting

It looks like you're speaking **Punjabi**!  

The phrase **"tussi ki kr rhe ho"** translates to **"What are you doing?"** in English.  

If you'd like any help with translations, learning Punjabi, or anything else, feel free to ask! 😊


#### Runnables
Following I give examples of usage of various types of runnables:

- RunnableLambda: to wrap any python function into a runnable

In [5]:
def pr(text : str) -> str:
    print(f"Used a runnable to print {text}")

runnable = RunnableLambda(pr)
runnable.invoke("\"Hello world!\"")

Used a runnable to print "Hello world!"


- RunnableParallel: chain various tasks to execute in parallel

In [6]:
def word_count_runnable(text : str) -> str:
    print(len(text.split()))

parallelRunnable = RunnableParallel(
    pr = pr , 
    word_count = word_count_runnable
)

parallelRunnable.invoke("Hello World! Testing 1 2 3 ....")

# Here both the funcitons were executed parallely and whatever they returned was retured as dict by the invoke call

Used a runnable to print Hello World! Testing 1 2 3 ....
7


{'pr': None, 'word_count': None}

- RunnableMap: same as RunnableParallel

In [7]:
runnable = RunnableMap(
    pr = pr,
    word_count = word_count_runnable
)

runnable.invoke("whats up")

Used a runnable to print whats up
2


{'pr': None, 'word_count': None}

- Chaining various runnables 

In [8]:
def add_three(num : int) -> int:
    return num + 3
def add_two(num : int) -> int:
    return num + 2
def add_one(num : int) -> int:
    return num + 1

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(add_two)
runnable_3 = RunnableLambda(add_three)

chain = runnable_1 | runnable_2 | runnable_3

print(chain.invoke(5))

11


#### Streaming : 
- Streaming is critical in making applications based on LLMs feel responsive to end-users.
- Basically here we print the tokens one by one as we get them instead of just printing then all together at the end to keep the user enagaged

In [11]:
for i in model.stream([{"role":"system" , "content": "what language do you think the user is speaking in? and then write the phrase in the native script of the language"} , {"role": "user", "content": "tussi ki kr rhe ho"}]):
    print(i.content,end="")

The phrase **"tussi ki kr rhe ho"** is in **Punjabi** (Gurmukhi script: **ਤੁਸੀਂ ਕੀ ਕਰ ਹੇ ਹੋ?**), which translates to **"What are you doing?"** in English.  

### Breakdown:
- **ਤੁਸੀਂ (tussī̃)** = You (formal)  
- **ਕੀ (kī)** = What  
- **ਕਰ ਹੇ ਹੋ (kar rahe ho)** = Are doing  

Let me know if you'd like any other details! 

#### Messages:
Here we can pass vraious types of mesasge to llm model to get better responses

In [10]:
message = [
    SystemMessage("You are a helpful ai chatbot whose task is to help the user in any way possible and be utmost polite with them"),
    HumanMessage("Hello how are you"),
    AIMessage("I am fine, how are you doing?"),
    HumanMessage("I am feeling sad today suggest something fun to me")
]

display(Markdown(model.invoke(message).content))

I'm sorry to hear you're feeling sad, but I'm here to help! Here are some fun and uplifting ideas to brighten your day:  

1. **Dance Party** – Put on your favorite upbeat songs and dance like nobody's watching!  
2. **Watch a Funny Movie/Show** – Laughter is the best medicine! Try classics like *The Office*, *Friends*, or *Monty Python*.  
3. **Creative Outlet** – Draw, paint, write a silly poem, or try DIY crafts (even if it’s messy!).  
4. **Virtual Adventure** – Take a free virtual tour of museums, zoos, or even the International Space Station!  
5. **Bake Something Sweet** – Cookies, brownies, or a simple mug cake can be delicious therapy.  
6. **Play with Pets (or Watch Cute Animal Videos)** – If you don’t have a pet, YouTube has endless adorable animal clips!  
7. **Gratitude List** – Write down 3 small things you’re grateful for—it can shift your mood.  
8. **Random Act of Kindness** – Compliment someone, send a nice text, or donate to a cause—it feels great!  

Would you like something more specific—like a game, a mindfulness exercise, or a feel-good playlist? I’m happy to tailor suggestions for you. 💛

#### Prompt Template:
- Most of the times the prompt we give to the ai is based on a tmeplate with multiple vairables or specific structure.
-  We can use this runnable to best manage messages

In [ ]:
systemTemplate = "You are a helpful assitant chatbot to assist the user in any way possible"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , systemTemplate) , 
        ("user" , "{text}"),
    ]
)

chain = LLMChain(prompt = prompt , llm = model)

for i in chain.stream("write 10 pointers on equality"): # sop this shows that we can't easily stream the output in a chain
    print(i['text'] , end="",flush=True)

/tmp/ipykernel_6356/3261266335.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt = prompt , llm = model)


Here are 10 key pointers on **equality**:  

1. **Equal Rights** – Every individual should have the same legal, social, and political rights, regardless of gender, race, religion, or background.  
2. **Non-Discrimination** – No one should face unfair treatment based on identity, disability, sexual orientation, or economic status.  
3. **Equal Opportunities** – Everyone should have access to education, jobs, healthcare, and resources to succeed.  
4. **Gender Equality** – Men and women should have equal rights, pay, and representation in all fields.  
5. **Social Justice** – Systems and policies should promote fairness and reduce inequalities in wealth and power.  
6. **Inclusion** – Society should embrace diversity and ensure marginalized groups have a voice.  
7. **Economic Equality** – Fair wages, wealth distribution, and opportunities should reduce income gaps.  
8. **Education for All** – Equal access to quality education helps break cycles of poverty and inequality.  
9. **Legal P

In [12]:
for i in model.stream(prompt.invoke("write 10 pointers on equality")):
    print(i.content, end="",flush=True)

Certainly! Here are 10 key pointers on **equality**:  

1. **Fundamental Right** – Equality is a basic human right, ensuring that every individual is treated fairly without discrimination.  

2. **Equal Opportunities** – Everyone should have access to the same opportunities in education, employment, healthcare, and social participation.  

3. **Non-Discrimination** – No one should face prejudice based on race, gender, religion, caste, disability, sexual orientation, or economic status.  

4. **Legal Protection** – Laws and policies must safeguard equality and penalize discriminatory practices.  

5. **Gender Equality** – Equal rights and respect for all genders, including fair wages, representation, and freedom from stereotypes.  

6. **Social Justice** – Addressing systemic inequalities to ensure marginalized groups receive fair treatment and support.  

7. **Economic Equality** – Reducing wealth gaps through fair wages, taxation, and social welfare programs.  

8. **Inclusive Represe

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , systemTemplate) , 
        ("user" , "{text}"),
        ("ai" , "{aitext}")
    ]
)

prompt.invoke({"text":"to rbo" , "aitext":"whatsup"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assitant chatbot to assist the user in any way possible', additional_kwargs={}, response_metadata={}), HumanMessage(content='to rbo', additional_kwargs={}, response_metadata={}), AIMessage(content='whatsup', additional_kwargs={}, response_metadata={})])